# Visualization of MS spectrum

In [1]:
%matplotlib inline

from pathlib import Path

from matplotlib import pyplot as plt,__version__ as plt_version
from pkg_resources import get_distribution  # Comes with setuptools.
from pyteomics import mzml, auxiliary

import ipywidgets as widgets
from IPython.display import clear_output

print(f"Matplotlib version : {plt_version}")
try:
    print(f"pyteomics version: {get_distribution('pyteomics').version}")
except:
    print("pyteomics version not found")

Matplotlib version : 3.1.1
pyteomics version: 4.1.2


Choosing the file is done programmatically as the file upload may not work due to the size
of the mzML files.

In [2]:
# Get the mzml file.
DATA_DIR = Path("..").resolve().joinpath("data/")

mzml_file = DATA_DIR.joinpath("1937004_Q1_5.mzML")

In [3]:
scans = []
mz_arrays = []
intensity_arrays = []
retention_times = []

In [4]:
# Get the MS1 spectra.
with mzml.read(str(mzml_file)) as reader:
    for spectrum in reader:
        if spectrum["ms level"] == 1:
            scans.append(spectrum["id"])
            mz_arrays.append(spectrum["m/z array"])
            intensity_arrays.append(spectrum["intensity array"])
            retention_times.append(spectrum["scanList"]["scan"][0]["scan start time"])

In [5]:
# Widget management.
file_box = widgets.Select(
    options=retention_times,
    # Writing Retention time is too long for the default label.
    description='RT (minutes):',
    disabled=False,
)

button = widgets.Button(description='Display',
                        button_style="info")


# Configuring the widgets interactions.

out = widgets.Output()

# Display the image when the button is clicked.  Is rather slow.
def display_image(_):
    with out:
        clear_output()
        # Supposes the retention time have same index as arrays.
        spectrum_index = retention_times.index(file_box.value)
        # The width needs to be adjusted for some arrays with extreme peaks.
        plt.bar(mz_arrays[spectrum_index], intensity_arrays[spectrum_index])
        plt.xlabel("m/z")
        plt.ylabel("intensity")
        plt.show()

button.on_click(display_image)

In [6]:
# Main widget.
box = widgets.VBox([file_box, button, out])
box

## Try using pyopenMS to parse the idXML

In [7]:
idxml_file = DATA_DIR.joinpath("1937004_Q1_5.idXML")

In [8]:
from pyopenms import IdXMLFile

In [9]:
protein_ids = []
peptide_ids = []

IdXMLFile().load(str(idxml_file), protein_ids, peptide_ids)

In [10]:
protein_ids
dir(protein_ids)

['__add__',
 '__class__',
 '__contains__',
 '__delattr__',
 '__delitem__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getitem__',
 '__gt__',
 '__hash__',
 '__iadd__',
 '__imul__',
 '__init__',
 '__init_subclass__',
 '__iter__',
 '__le__',
 '__len__',
 '__lt__',
 '__mul__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__reversed__',
 '__rmul__',
 '__setattr__',
 '__setitem__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 'append',
 'clear',
 'copy',
 'count',
 'extend',
 'index',
 'insert',
 'pop',
 'remove',
 'reverse',
 'sort']

In [11]:
help(peptide_ids)

Help on list object:

class list(object)
 |  list(iterable=(), /)
 |  
 |  Built-in mutable sequence.
 |  
 |  If no argument is given, the constructor creates a new empty list.
 |  The argument must be an iterable if specified.
 |  
 |  Methods defined here:
 |  
 |  __add__(self, value, /)
 |      Return self+value.
 |  
 |  __contains__(self, key, /)
 |      Return key in self.
 |  
 |  __delitem__(self, key, /)
 |      Delete self[key].
 |  
 |  __eq__(self, value, /)
 |      Return self==value.
 |  
 |  __ge__(self, value, /)
 |      Return self>=value.
 |  
 |  __getattribute__(self, name, /)
 |      Return getattr(self, name).
 |  
 |  __getitem__(...)
 |      x.__getitem__(y) <==> x[y]
 |  
 |  __gt__(self, value, /)
 |      Return self>value.
 |  
 |  __iadd__(self, value, /)
 |      Implement self+=value.
 |  
 |  __imul__(self, value, /)
 |      Implement self*=value.
 |  
 |  __init__(self, /, *args, **kwargs)
 |      Initialize self.  See help(type(self)) for accurate sign

In [12]:
pep_ids = list(peptide_ids)
print(len(pep_ids))

31920


In [13]:
print(pep_ids[0].getMZ())
print(pep_ids[0].getMetaValue("spectrum_reference"))

415.205402217913
b'controllerType=0 controllerNumber=1 scan=1008'


In [14]:
for hit in pep_ids[0].getHits():
    print(" - Peptide hit rank:", hit.getRank())
    print(" - Peptide hit sequence:", hit.getSequence().toString())
    print(" - Peptide hit score:", hit.getScore())

 - Peptide hit rank: 0
 - Peptide hit sequence: b'VQPEDNK'
 - Peptide hit score: 0.8249030113220215
